In [ ]:
!pip install transformers
!pip install tensorflow
!pip install torch
!pip install pytorch

In [1]:
from transformers import AutoTokenizer, T5ForConditionalGeneration,T5EncoderModel
import os
import tensorflow as tf
import torch
import pickle
import gc
from torch.nn import MaxPool2d
import glob
import re
from transformers import logging
from config import *
from lib import *

2023-07-14 14:46:47.511383: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-14 14:46:47.584912: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-14 14:46:49.891583: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
logging.set_verbosity_error()

device = "cuda" if torch.cuda.is_available() else "cpu"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = T5EncoderModel.from_pretrained(MODEL_NAME)
model = model.to(device)

In [3]:
device

'cpu'

In [ ]:
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = 'max_split_size_mb:512'
os.environ["TOKENIZERS_PARALLELISM"] = 'true'

In [ ]:
# Initialize exception flag
exception = False

# Iterate through folders in the specified Java files path
for folder in sorted(os.listdir(JAVA_FILES_PATH)):
    folder_path = JAVA_FILES_PATH + "/" + folder
    
    # Skip certain file types
    if '.csv' in folder or '.ipynb' in folder or '.txt' in folder:
        continue
    
    print(folder)
    
    # Iterate through versions in the folder
    for version in os.listdir(folder_path):
        print(version)
        version_path = folder_path + "/" + version
        output_full_path = f'{version_path}/{OUTPUT_PATH}'
        
        # Skip if output path already exists
        if os.path.isdir(output_full_path):
            continue
        
        try:
            # Create the output path directory if it doesn't exist
            os.mkdir(output_full_path)
        except OSError as error: 
            print(error)
        
        # Process each Java file in the version path
        for filename in glob.iglob(f'{JAVA_FILES_PATH}/{folder}/{version}' + '/**/*.java', recursive=True):
            if os.path.isdir(filename):
                print(f"{filename} is a directory")
                continue
            
            # Extract the file name
            file = filename.split('/')[-1].split('.')[0]
            
            try:
                # Attempt to create an embedding from the file
                model = create_embedding_from_file(filename, model, version_path)
            except RuntimeError:
                # Handle runtime error (e.g., CUDA out of memory)
                exception = True
                del input_ids
                torch.cuda.empty_cache() 
                gc.collect()
                pass

            if exception:
                print(f'{version} {file} - the file cannot be encoded using the model')
                model = T5EncoderModel.from_pretrained(MODEL_NAME)
                model = model.to(device)
                exception = False
                continue